In [185]:
def _make_report_header(maxN):
    """Return the header and format string for reporting coverage."""
    nw = len(str(maxN))
    frac_width = 2*nw + 1  # width of 300/1000
    frac_header = 'Sample'
    frac_format = '{i:' + str(nw) + 'd}/{n:' + str(nw) + 'd}'
    if frac_width < len(frac_header):
        pad = ''.join([' ' for _ in range(len(frac_header) - frac_width)])
        frac_format = pad + frac_format
    elif len(frac_header) < frac_width:
        pad = ''.join([' ' for _ in range(frac_width - len(frac_header))])
        frac_header = pad + frac_header
    hdr = frac_header + "   Coverage   MinGap   Median   MaxGap     Elapsed     Remaining   Infeasible"
    fmt = frac_format + "    {cov:6.2f}%   {min:6.4f}   {med:6.4f}   {max:6.4f}   {ela:9.2f}     {rem:9.2f}   {inf:10d}"
    return hdr, fmt



In [186]:
def _maxgap(points, fva=None):
    # points has rows = samples, columns = variables

    # make a copy because we're going to sort the columns
    points = points.copy()
    if fva is not None:
        points = np.vstack((fva.minimum, points, fva.maximum))
    points.sort(0)

    gaps = points[1:,:] - points[0:-1,:]
    width = gaps.max(0)
    loc = gaps.argmax(0)
    left = np.zeros(width.size)
    for i in range(width.size):
        left[i] = points[loc[i],i]
    relative = width / (points[-1,:] - points[0,:])
    target = left + width/2

    return relative, target, width



In [188]:
def _generate_sample(
        primary_var, primary_lb, primary_ub,
        secondary_vars=None, secondary_targets=None, secondary_weights=None):

    #Create a model
    #from pyomo.environ import sin, cos
    model = ConcreteModel()
    model.IDX2 = range(bus_len)
    model.t = Var(model.IDX2)

    model.vars = Var(range(len(fva)))
    model.vars[primary_var].setlb(primary_lb)
    model.vars[primary_var].setub(primary_ub)


    if secondary_vars is not None:
        quad_exp = 0
        model.c = ConstraintList()
        
        #Bus voltage angle initial condition
        model.c.add(model.t[0] == 0)

        for i, sec in enumerate(secondary_vars):
            quad_exp += secondary_weights[i] * (model.vars[sec]-secondary_targets[i])**2
        
        #Limits constraints
        base = 100
        for i in range(gen_len):
            model.c.add(expr = model.vars[i] <= gen['Pmax'][i]/base)
            model.c.add(expr = model.vars[i] >= gen['Pmin'][i]/base)
            model.c.add(expr = model.vars[i+5] <= gen['Qmax'][i]/base)
            model.c.add(expr = model.vars[i+5] >= gen['Qmin'][i]/base)
        for i in range(bus_len):    
            model.c.add(expr = model.vars[i+10] <= bus['Vmax'][i])
            model.c.add(expr = model.vars[i+10] >= bus['Vmin'][i])
        
        #Nodal equations
        count = 0
        for i in range(bus_len):
            if (np.any(gen['bus'].ravel()-1==[i])):
                holder1 = model.vars[count]
                holder2 = model.vars[count+5]
                count = count+1
            else:
                holder1 = 0
                holder2 = 0

            model.c.add(expr = sum([model.vars[i+10]*model.vars[j+10]*cmath.polar(Ybus[i,j])[0]*cos(model.t[i]-model.t[j]-cmath.polar(Ybus[i,j])[1]) for j in range(bus_len)]) - holder1 + Pd[i]== 0)
            model.c.add(expr = sum([model.vars[i+10]*model.vars[j+10]*cmath.polar(Ybus[i,j])[0]*sin(model.t[i]-model.t[j]-cmath.polar(Ybus[i,j])[1]) for j in range(bus_len)]) - holder2 + Qd[i]== 0)

        #Line flow constraints
        
        for i in range(len(branch)): 
            x = int(branch.loc[i]['fbus'])-1
            y = int(branch.loc[i]['tbus'])-1
            val = (branch.loc[i]['rateA']/base)
            if(val == 0):
                val = 100
        #    With phasor
            Pxy = (model.vars[x+10]**2)*cmath.polar(Ybus[x,y])[0]*cos(cmath.polar(Ybus[x,y])[1]) - model.vars[x+10]*model.vars[y+10]*cmath.polar(Ybus[x,y])[0]*cos(model.t[x]-model.t[y]-cmath.polar(Ybus[x,y])[1])
            Qxy =-(model.vars[x+10]**2)*cmath.polar(Ybus[x,y])[0]*sin(cmath.polar(Ybus[x,y])[1]) - model.vars[x+10]*model.vars[y+10]*cmath.polar(Ybus[x,y])[0]*sin(model.t[x]-model.t[y]-cmath.polar(Ybus[x,y])[1])     

            Pyx = (model.vars[y+10]**2)*cmath.polar(Ybus[x,y])[0]*cos(cmath.polar(Ybus[x,y])[1]) - model.vars[x+10]*model.vars[y+10]*cmath.polar(Ybus[x,y])[0]*cos(model.t[y]-model.t[x]-cmath.polar(Ybus[x,y])[1])
            Qyx =-(model.vars[y+10]**2)*cmath.polar(Ybus[x,y])[0]*sin(cmath.polar(Ybus[x,y])[1]) - model.vars[x+10]*model.vars[y+10]*cmath.polar(Ybus[x,y])[0]*sin(model.t[y]-model.t[x]-cmath.polar(Ybus[x,y])[1])     

            model.c.add(expr = (Pxy**2 + Qxy**2)<= val**2)
            model.c.add(expr = (Pyx**2 + Qyx**2)<= val**2)


        model.cost = Objective(expr = quad_exp)
    else:
        model.cost = Objective(expr = 0)

    solution = SolverFactory('ipopt').solve(model)
    if str(solution['Solver'].Termination_condition) != 'optimal':
        return None
    else:
        result_v = np.array([model.vars[i].value for i in range(len(fva))])
#         result_t = np.array([model.t[i].value for i in range(bus_len)])
        return result_v#, result_t


In [189]:
import time

import numpy as np
import pandas as pd
import cmath
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
from pyomo.environ import *


#load data
xls = pd.ExcelFile('data\case14.xlsx')
branch = pd.read_excel(xls, 'branch')
gen = pd.read_excel(xls, 'gen')
bus = pd.read_excel(xls, 'bus')
cost = pd.read_excel(xls, 'cost')
base = 100  

#Y matrix
base = 100
bus_len = bus.shape[0]
gen_len = len(gen['bus'].ravel())

Ybus = np.matrix(np.zeros([bus_len,bus_len]),dtype=complex)
shunt = np.zeros(bus_len, dtype=complex)
x = [pd.DataFrame() for i in range(bus_len)]
for i in range(bus_len):  
    x[i] = branch.loc[branch['fbus'] == i]
    fro = [j for j in x[i]['tbus']]
    fro = pd.DataFrame(fro)
    for j in range(len(fro)):
        Ybus[i-1,fro.loc[j][0]-1] =  -1/complex(pd.DataFrame(x[i]['r']).iloc[j][0],pd.DataFrame(x[i]['x']).iloc[j][0])
        Ybus[fro.loc[j][0]-1,i-1] =  -1/complex(pd.DataFrame(x[i]['r']).iloc[j][0],pd.DataFrame(x[i]['x']).iloc[j][0])
        shunt[j] = complex(0,sum(x[i]['b'])/2)

for i in range(bus_len):
    Ybus[i,i] = np.sum(Ybus[i,:])*(-1) + complex(0,sum(branch.loc[(branch['fbus']==i+1) | (branch['tbus']==i+1)]['b'])/2)


# declare variables
Pd = np.zeros(bus_len)
Qd = np.zeros(bus_len)
Vmax = np.zeros(bus_len)
Vmin = np.zeros(bus_len)
Pmax = np.zeros(bus_len)
Pmin = np.zeros(bus_len)
Qmax = np.zeros(bus_len)
Qmin = np.zeros(bus_len)
cost_lin = np.zeros(bus_len)
cost_quad = np.zeros(bus_len)

for i in range(bus_len):
    Pd[i] = bus['Pd'][i]/base
    Qd[i] = bus['Qd'][i]/base
    Vmax[i] = bus['Vmax'][i]
    Vmin[i] = bus['Vmin'][i]

for i in range(gen_len):
    k = i
    i = gen['bus'].ravel()[i]
    Pmax[i-1] = gen.loc[gen['bus'] == i]['Pmax'].ravel()[0]/base
    Pmin[i-1] = gen.loc[gen['bus'] == i]['Pmin'].ravel()[0]/base
    Qmax[i-1] = gen.loc[gen['bus'] == i]['Qmax'].ravel()[0]/base
    Qmin[i-1] = gen.loc[gen['bus'] == i]['Qmin'].ravel()[0]/base
    cost_lin[i-1] = cost['lin'][k]*base
    cost_quad[i-1] = cost['quad'][k]*base**2
# 

def gapsplit(
        n, max_tries = None,
        primary='sequential', primary_tol=0.001,
        secondary_frac=0.05,
        min_range=1e-5,
        enforce_range=True,
        report_interval=0.1,
        quiet=False):
    
    if quiet:
        report = lambda s: None
    else:
        report = lambda s: print(s)

    p = pd.DataFrame([gen['Pmin'],gen['Pmax']]).T
    p.columns = ['minimum', 'maximum']
    p = p.T
    p.columns = ['P1','P2','P3','P4','P5']
    p = p.T/base

    q = pd.DataFrame([gen['Qmin'],gen['Qmax']]).T
    q.columns = ['minimum', 'maximum']
    q = q.T
    q.columns = ['Q1','Q2','Q3','Q4','Q5']
    q = q.T/base

    v = pd.DataFrame([bus['Vmin'],bus['Vmax']]).T
    v.columns = ['minimum', 'maximum']
    v = v.T
    v.columns = ['V1','V2','V3','V4','V5','V6','V7',
                 'V8','V9','V10','V11','V12','V13','V14']
    v = v.T

    frames = [p,q,v]
    fva = pd.concat(frames)

    if secondary_frac >= 1.0:
        n_secondary = secondary_frac
    else:
        n_secondary = np.floor(secondary_frac * len(fva)).astype(int)

    # only split reactions with feasible range >= min_range
    idxs = (fva.maximum - fva.minimum >= min_range).to_numpy().nonzero()[0]
    weights = (1/(fva.maximum - fva.minimum)**2).to_numpy()
    report("Targeting {}/{} unblocked primary variables.".format(len(idxs), len(fva)))
    report("Targeting {} secondary variables.".format(n_secondary))

    report_header, report_format = _make_report_header(n)
    report("\n" + report_header)
    if report_interval < 1.0:
        report_interval = np.floor(report_interval * n).astype(int)
    samples = np.zeros((n, len(fva)))
#     angles = np.zeros((n,bus_len))

    k = 0
    infeasible_count = 0

    primary_var = -1
    try_ = 0
    start_time = time.time()
    
    while True:
        if max_tries is not None and try_ >= max_tries:
            break
        try_ += 1
        relative, target, width = _maxgap(samples[0:k,idxs], fva.iloc[idxs,:])
        #primary_var = np.argmax(relative)
        primary_var += 1
        if primary_var >= len(idxs):
            primary_var = 0

        primary_target = target[primary_var]
        primary_lb = primary_target - primary_tol*width[primary_var]
        primary_ub = primary_target + primary_tol*width[primary_var]
        secondary_vars = np.random.choice(len(idxs), n_secondary, replace=False)
        secondary_targets = target[secondary_vars]
        secondary_weights = weights[idxs[secondary_vars]]

        #model = pyo.ConcreteModel()
        new_sample =  _generate_sample(
                    idxs[primary_var], primary_lb, primary_ub,
                    idxs[secondary_vars], secondary_targets, secondary_weights)


        if new_sample is not None:
            if enforce_range:
                new_sample[new_sample > fva.maximum] = fva.maximum[new_sample > fva.maximum]
                new_sample[new_sample < fva.minimum] = fva.minimum[new_sample < fva.minimum]

            samples[k,:] = new_sample
#             angles[k,:] = angles_
            k += 1
            if k % report_interval == 0:
                elapsed = time.time() - start_time
                remaining = elapsed / k * (n - k)
                report(report_format.format(
                        i=k, n=n, cov=100*(1-np.mean(relative)),
                        min=np.min(relative), med=np.median(relative),
                        max=np.max(relative), ela=elapsed, rem=remaining,
                        inf=infeasible_count))
        else:
            infeasible_count += 1

        if k >= n: break
    
    if k < n:
        # max_tries reached; return fewer than n samples
        samples = samples[:k,:]
#         angles = angles[:k,:]

    return pd.DataFrame(data=samples,columns=fva.maximum.index)#, pd.DataFrame(angles)
    
    #sol = str(SolverFactory('ipopt').solve(model)['Solver'].Termination_condition)

In [190]:
ans = gapsplit(5)

Targeting 24/24 unblocked primary variables.
Targeting 1 secondary variables.

Sample   Coverage   MinGap   Median   MaxGap     Elapsed     Remaining   Infeasible
   1/5      0.00%   1.0000   1.0000   1.0000        0.11          0.45            0


C:\Users\abuga\Anaconda3\lib\site-packages\ipykernel_launcher.py:161: RuntimeWarning: divide by zero encountered in long_scalars


    model=unknown;
        message from solver=Ipopt 3.11.1\x3a Converged to a locally infeasible
        point. Problem may be infeasible.
   2/5     32.52%   0.5000   0.6751   0.9132        0.47          0.70            1
   3/5     40.99%   0.3407   0.5593   0.8383        0.58          0.39            1
   4/5     49.68%   0.3229   0.5003   0.8036        0.73          0.18            1
   5/5     53.09%   0.2735   0.4615   0.7920        0.85          0.00            1


In [191]:
min_range = 1e-5
secondary_frac=0.15
report = lambda s: print(s)
n = 100 

p = pd.DataFrame([gen['Pmin'],gen['Pmax']]).T
p.columns = ['minimum', 'maximum']
p = p.T
p.columns = ['P1','P2','P3','P4','P5']
p = p.T/base

q = pd.DataFrame([gen['Qmin'],gen['Qmax']]).T
q.columns = ['minimum', 'maximum']
q = q.T
q.columns = ['Q1','Q2','Q3','Q4','Q5']
q = q.T/base

v = pd.DataFrame([bus['Vmin'],bus['Vmax']]).T
v.columns = ['minimum', 'maximum']
v = v.T
v.columns = ['V1','V2','V3','V4','V5','V6','V7',
             'V8','V9','V10','V11','V12','V13','V14']
v = v.T

frames = [p,q,v]
fva = pd.concat(frames)

n_secondary = np.floor(secondary_frac * len(fva)).astype(int)

idxs = (fva.maximum - fva.minimum >= min_range).to_numpy().nonzero()[0]
weights = (1/(fva.maximum - fva.minimum)**2).to_numpy()

report("Targeting {}/{} unblocked primary variables.".format(len(idxs), len(fva)))
report("Targeting {} secondary variables.".format(n_secondary))

report_header, report_format = _make_report_header(n)
report("\n" + report_header)

samples = np.zeros((n, len(fva)))

relative, target, width = _maxgap(samples[0:k,idxs], fva.iloc[idxs,:])
primary_tol=0.001
primary_var = 0
primary_target = target[primary_var]
primary_lb = primary_target - primary_tol*width[primary_var]
primary_ub = primary_target + primary_tol*width[primary_var]
secondary_vars = np.random.choice(len(idxs), n_secondary, replace=False)
secondary_targets = target[secondary_vars]
secondary_weights = weights[idxs[secondary_vars]]

# model = pyo.ConcreteModel()

d  =  _generate_sample(
                idxs[primary_var], primary_lb, primary_ub,
                idxs[secondary_vars], secondary_targets, secondary_weights)

Targeting 24/24 unblocked primary variables.
Targeting 3 secondary variables.

 Sample   Coverage   MinGap   Median   MaxGap     Elapsed     Remaining   Infeasible


In [193]:
ans

,P1,P2,P3,P4,P5,Q1,Q2,Q3,Q4,Q5,...,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14
0,1.661972e+00,0.195257,0.225971,0.318528,0.260947,0.047675,0.173869,0.269241,0.203432,0.085993,...,1.011959,1.000081,0.985924,1.000000,0.968423,0.965777,0.978750,0.983002,0.976388,0.952266
1,5.347388e-01,0.436948,0.612985,0.659264,0.368386,0.048249,0.018064,0.210432,0.181087,0.149796,...,1.017091,1.007625,0.996542,1.020370,0.975530,0.972624,0.985526,0.990803,0.983643,0.959400
2,2.792310e-01,0.500915,0.547350,0.829632,0.452903,0.038205,-0.086114,0.258112,0.229313,0.166164,...,1.047671,1.047270,1.029508,1.054482,1.009434,1.007647,1.022763,1.030680,1.023053,0.996337
3,4.558841e-01,0.455261,0.441364,0.530382,0.726451,0.048558,0.149360,0.252674,0.060543,0.188254,...,1.025761,0.990504,0.997894,1.022447,0.976822,0.971135,0.976882,0.974580,0.969370,0.954008
4,4.620686e-07,0.787914,0.446415,0.860714,0.517290,0.074228,-0.247754,0.399997,0.240000,0.160713,...,1.055720,1.056422,1.036862,1.060000,1.017677,1.016137,1.031633,1.039892,1.032261,1.005181


In [194]:
for k in range(len(ans)):
    print(np.all([
        np.all([ans.loc[k][i] <= gen['Pmax'][i]/base for i in range(5)]) ,
        np.all([ans.loc[k][i] >= gen['Pmin'][i]/base for i in range(5)]) ,
        np.all([ans.loc[k][i+5] <= gen['Qmax'][i]/base for i in range(5)]) ,
        np.all([ans.loc[k][i+5] >= gen['Qmin'][i]/base for i in range(5)]) ,
        np.all([ans.loc[k][i+10] <= bus['Vmax'][i] for i in range(14)]) ,
        np.all([ans.loc[k][i+10] >= bus['Vmin'][i] for i in range(14)]) ,

        ]))

True
True
True
True
True


In [ ]:
for k in range(len(branch)): 
    x = int(branch.loc[k]['fbus'])-1
    y = int(branch.loc[k]['tbus'])-1
    p1[k] = abs((((vv[x])**2)*cmath.polar(Ybus[x,y])[0])*np.cos(cmath.polar(Ybus[x,y])[1]) - (vv[x])*(vv[y])*cmath.polar(Ybus[x,y])[0]*np.cos((tt[x])-(tt[y])-cmath.polar(Ybus[x,y])[1]))<= (branch.loc[k]['rateA'])/base

#Check active/reactive power balance
for k in range(bus_len):
    p2[k] = sum([(vv[k])*(vv[j])*cmath.polar(Ybus[k,j])[0]*np.cos((tt[k])-(tt[j])-cmath.polar(Ybus[k,j])[1]) for j in range(bus_len)]) - pv[k] + Pd[k] <=0.001
    p3[k] = sum([(vv[k])*(vv[j])*cmath.polar(Ybus[k,j])[0]*np.sin((tt[k])-(tt[j])-cmath.polar(Ybus[k,j])[1]) for j in range(bus_len)]) - qv[k] + Qd[k] <=0.001

prop1 = np.all(p1)
prop2 = np.all(p2)
prop3 = np.all(p3)

prop[i] = prop1 & prop2 & prop3
if(prop[i]):
    counter = counter + 1